In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from scipy.stats import pearsonr

from zp_ihlt_project.config import TRAIN_DATA_WITH_FEATURES_PATH, TEST_DATA_WITH_FEATURES_PATH, FEATURE_STEPS_PATH, FEATURE_STEPS_WITH_IMPORTANCE_PATH
from zp_ihlt_project.feature_extraction import lexical_functions, preprocessing_functions, semantic_functions, ngram_functions, sentence_to_doc


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/zachparent/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/zachparent/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Train and Evaluate Models

This notebook demonstrates random forest regression models trained on various sets of features which were extracted in [extract_features.ipynb](extract_features.ipynb).


### Load feature data

We load the features that were extracted in [extract_features.ipynb](extract_features.ipynb), for both the training and test datasets. The test dataset is only used for final evaluation, and never for training. Validation is performed using cross-validation, as well as a train/val split.

In [2]:
all_train_df = pd.read_csv(TRAIN_DATA_WITH_FEATURES_PATH)
all_test_df = pd.read_csv(TEST_DATA_WITH_FEATURES_PATH)
feature_steps = pd.read_csv(FEATURE_STEPS_PATH)
feature_names = [col for col in all_train_df.columns if col.startswith("score_")]

## Baseline Model

We start by training a model on all features, using 5-fold cross-validation. We use a custom scorer function to calculate Pearson correlation with the gold standard scores.


In [3]:
results = []

def pearsonr_score(estimator, X, y):
    y_pred = estimator.predict(X)
    return pearsonr(y, y_pred)[0]

model = RandomForestRegressor()

scores = cross_val_score(
    model, 
    all_train_df[feature_names],
    all_train_df.gs,
    cv=5,
    n_jobs=-1,
    scoring=pearsonr_score
)
print(f"Cross-validation Pearson: {np.mean(scores)}+-{np.std(scores)}")


Cross-validation Pearson: 0.7338617021984515+-0.10283672501615145


We see that the model achieves a Pearson correlation of ~0.730, with a rather large standard deviation of ~0.10.

In [4]:
train_idx, val_idx = train_test_split(range(len(all_train_df)), test_size=0.2, random_state=42)

In [5]:
model = RandomForestRegressor()
model.fit(all_train_df.iloc[train_idx][feature_names], all_train_df.iloc[train_idx].gs)


RandomForestRegressor()

### Datasets
Sometimes we will evaluate the model on all datasets, but sometimes we will evaluate on a single corpus at a time.

In [6]:
train_datasets = all_train_df.dataset.unique().tolist()
test_datasets = all_test_df.dataset.unique().tolist()

### Evaluation Helper

We will be training models which vary by the features used, we will need to split data by dataset, and we will need to evaluate them across train/val/test. We define a helper function to get the `X` and `y` data given these parameters.

Our evaluation helper function also calculates Pearson correlation with the gold standard scores. We can see the scores for each split and each dataset, as well as the overall scores.

In [7]:
def get_X_y(df, selected_features, dataset, idx=None):
    if idx is None:
        idx = range(len(df))
    X = df.iloc[idx]
    X = X.loc[X.dataset == dataset][selected_features]
    y = df.iloc[idx]
    y = y.loc[y.dataset == dataset].gs
    return X, y

def evaluate_model_across_train_val_test(model, selected_features):
    results = []
    datasets = train_datasets
    for dataset in datasets:
        dataset_results = []
        dataset_results.append(dataset)
        
        X_train, y_train = get_X_y(all_train_df, selected_features, dataset, train_idx)
        X_val, y_val = get_X_y(all_train_df, selected_features, dataset, val_idx)
        X_test, y_test = get_X_y(all_test_df, selected_features, dataset)

        preds = model.predict(X_train)
        dataset_results.append(pearsonr(y_train, preds)[0])

        preds = model.predict(X_val)
        dataset_results.append(pearsonr(y_val, preds)[0])

        preds = model.predict(X_test)
        dataset_results.append(pearsonr(y_test, preds)[0])

        results.append(dataset_results)

    dataset_results = []
    dataset_results.append('all')
    preds = model.predict(all_train_df.iloc[train_idx][selected_features])
    dataset_results.append(pearsonr(all_train_df.iloc[train_idx].gs, preds)[0])

    preds = model.predict(all_train_df.iloc[val_idx][selected_features])
    dataset_results.append(pearsonr(all_train_df.iloc[val_idx].gs, preds)[0])

    preds = model.predict(all_test_df[all_test_df.dataset.isin(train_datasets)][selected_features])
    dataset_results.append(pearsonr(all_test_df[all_test_df.dataset.isin(train_datasets)].gs, preds)[0])

    results.append(dataset_results)

    results = pd.DataFrame(results, columns=["dataset", "train_pearson", "val_pearson", "test_pearson"])
    return results

evaluate_model_across_train_val_test(model, feature_names)

,dataset,train_pearson,val_pearson,test_pearson
0,MSRpar,0.973558,0.655917,0.649252
1,MSRvid,0.981976,0.766652,0.843264
2,SMTeuroparl,0.964678,0.780336,0.494888
3,all,0.984913,0.842813,0.810730


With no feature selection, we see that the model achieves a Pearson correlation of ~0.984 on the training set, ~0.840 on the validation set, and ~0.808 on the test set.

## Analyze Feature Importances

\>2000 features is a lot of features! It requires ~1min to train, and it may not be clear which features the model is paying attention to. A random forest regressor allows us to extract the feature importances directly and inspect the values. We can then try training and evaluating new models on a subset of features.

In [8]:
feature_importances = model.feature_importances_
ranked_feature_importance_indices = np.argsort(feature_importances)[::-1]
feature_steps['importance'] = feature_importances
feature_steps.sort_values(by='importance', ascending=False, inplace=True)
feature_steps.to_csv(FEATURE_STEPS_WITH_IMPORTANCE_PATH)
feature_steps.head(10)

,name,metric,step_0,step_1,step_2,step_3,step_4,step_5,step_6,step_7,importance
660,score_jaccard_165,jaccard,sentence_to_doc,get_tokens,remove_stopwords,lemmatize_tokens,get_characters,get_2grams,NaN,NaN,0.165530
1029,score_cosine_257,cosine,sentence_to_doc,chunk_NEs,remove_stopwords,lemmatize_tokens,get_characters,get_2grams,NaN,NaN,0.097958
661,score_cosine_165,cosine,sentence_to_doc,get_tokens,remove_stopwords,lemmatize_tokens,get_characters,get_2grams,NaN,NaN,0.069857
1032,score_jaccard_258,jaccard,sentence_to_doc,chunk_NEs,remove_stopwords,lemmatize_tokens,get_characters,get_3grams,NaN,NaN,0.031651
1028,score_jaccard_257,jaccard,sentence_to_doc,chunk_NEs,remove_stopwords,lemmatize_tokens,get_characters,get_2grams,NaN,NaN,0.026341
1033,score_cosine_258,cosine,sentence_to_doc,chunk_NEs,remove_stopwords,lemmatize_tokens,get_characters,get_3grams,NaN,NaN,0.022583
656,score_jaccard_164,jaccard,sentence_to_doc,get_tokens,remove_stopwords,lemmatize_tokens,get_characters,NaN,NaN,NaN,0.020005
1024,score_jaccard_256,jaccard,sentence_to_doc,chunk_NEs,remove_stopwords,lemmatize_tokens,get_characters,NaN,NaN,NaN,0.019760
1037,score_cosine_259,cosine,sentence_to_doc,chunk_NEs,remove_stopwords,lemmatize_tokens,get_characters,get_4grams,NaN,NaN,0.018566
388,score_jaccard_97,jaccard,sentence_to_doc,chunk_NEs,lemmatize_tokens,get_characters,get_2grams,NaN,NaN,NaN,0.017382


In [9]:
print(f"Sum of top 10 feature importances: {sum(feature_steps.head(10).importance)}")
print(f"Sum of all feature importances: {sum(feature_steps.importance)}")
print(f"Number of features: {len(feature_steps)}")

Sum of top 10 feature importances: 0.4896318698073132
Sum of all feature importances: 1.0000000000000002
Number of features: 2080


We see that the top 10 features account for ~50% of the total feature importances.

### Train Models on features with non-zero importances
We can try training models on a subset of the features with non-zero importances.

In [10]:
nonzero_feature_importances = feature_importances > 0
nonzero_importance_feature_indexes = ranked_feature_importance_indices[nonzero_feature_importances]
nonzero_importance_features = np.array(feature_names)[nonzero_importance_feature_indexes]
print(f"Number of features with non-zero importances: {len(nonzero_importance_features)}")
nonzero_importance_features_model = RandomForestRegressor()
nonzero_importance_features_model.fit(all_train_df.iloc[train_idx][nonzero_importance_features], all_train_df.iloc[train_idx].gs)

Number of features with non-zero importances: 1248


RandomForestRegressor()

It turns out there are 1248 features with non-zero importances, which is about 60% of the total number of features.

In [11]:
evaluate_model_across_train_val_test(nonzero_importance_features_model, nonzero_importance_features)

,dataset,train_pearson,val_pearson,test_pearson
0,MSRpar,0.974802,0.634937,0.645730
1,MSRvid,0.981573,0.773686,0.841937
2,SMTeuroparl,0.967723,0.792927,0.497677
3,all,0.984888,0.844559,0.810626


As expected, the difference in performance is negligible.

### Train Models on top 500 features
We can also try training models on the top 500 features.

In [12]:
top_500_features_indices = ranked_feature_importance_indices[:500]
top_500_features = np.array(feature_names)[top_500_features_indices]

In [13]:
top_500_features_model = RandomForestRegressor()
top_500_features_model.fit(all_train_df.iloc[train_idx][top_500_features], all_train_df.iloc[train_idx].gs)

RandomForestRegressor()

In [14]:
evaluate_model_across_train_val_test(top_500_features_model, top_500_features)

,dataset,train_pearson,val_pearson,test_pearson
0,MSRpar,0.973078,0.641725,0.638301
1,MSRvid,0.981953,0.766991,0.840068
2,SMTeuroparl,0.964748,0.784183,0.500648
3,all,0.984824,0.841767,0.809742


With only 500 features, we see a slight drop in performance, but the model takes about half the time to train.

## Evaluating Feature Groups

The methods used to extract features include lexical, semantic, and n-gram methods, along with some preprocessing steps. We can try training models on subsets of these feature groups to see if one is more important than the others.

We define a helper function to filter features by the steps used to extract them. We can specify the feature groups simply using the list of processing functions that are defined in [feature_extraction.py](../src/zp_ihlt_project/feature_extraction.py).

In [15]:
def filter_features_by_steps(
    feature_steps,
    allowed_steps,
):
    allowed_step_names = [step.__name__ for step in [sentence_to_doc, *allowed_steps]]
    step_col_names = [col for col in feature_steps.columns if col.startswith("step_")]
        
    selected_features = []
    for _, row in feature_steps.iterrows():
        if all(pd.isna(step) or step in allowed_step_names for step in row[step_col_names]):
            feature_name = row["name"]
            selected_features.append(feature_name)
    return selected_features


### Lexical Features

In [16]:
lexical_features = filter_features_by_steps(feature_steps, preprocessing_functions + lexical_functions + ngram_functions)
print(f"Number of lexical-based features: {len(lexical_features)}")

Number of lexical-based features: 320


In [17]:
lexical_model = RandomForestRegressor()
lexical_model.fit(all_train_df.iloc[train_idx][lexical_features], all_train_df.iloc[train_idx].gs)
evaluate_model_across_train_val_test(lexical_model, lexical_features)

,dataset,train_pearson,val_pearson,test_pearson
0,MSRpar,0.973639,0.626999,0.636883
1,MSRvid,0.981740,0.742171,0.814264
2,SMTeuroparl,0.962587,0.751219,0.522465
3,all,0.984528,0.826879,0.802228


Training using only Lexical Features and N-grams has a very slight detrimental effect on performance, but the model is still very strong. This suggests that the lexical features are an important component of the model, but not the only important component.

### Semantic Features

In [18]:
semantic_features = filter_features_by_steps(feature_steps, preprocessing_functions + semantic_functions + ngram_functions)
print(f"Number of semantic-based features: {len(semantic_features)}")

Number of semantic-based features: 240


In [19]:
semantic_model = RandomForestRegressor()
semantic_model.fit(all_train_df.iloc[train_idx][semantic_features], all_train_df.iloc[train_idx].gs)
evaluate_model_across_train_val_test(semantic_model, semantic_features)

,dataset,train_pearson,val_pearson,test_pearson
0,MSRpar,0.965303,0.539456,0.531373
1,MSRvid,0.966727,0.756345,0.805725
2,SMTeuroparl,0.960196,0.829197,0.421006
3,all,0.977184,0.831711,0.765633


If we use only semantic features, there is a fairly large drop in performance on the test set, although the validation set performance actually improves. This suggests that semantic features may not be very good generalizable features for this task.

### N-grams

We were also curious to see if n-grams were important, so we trained a model without them.

In [20]:
without_ngrams_features = filter_features_by_steps(feature_steps, preprocessing_functions + semantic_functions + lexical_functions)
print(f"Number of n-gram-based features: {len(without_ngrams_features)}")

Number of n-gram-based features: 520


In [21]:
without_ngrams_model = RandomForestRegressor()
without_ngrams_model.fit(all_train_df.iloc[train_idx][without_ngrams_features], all_train_df.iloc[train_idx].gs)
evaluate_model_across_train_val_test(without_ngrams_model, without_ngrams_features)

,dataset,train_pearson,val_pearson,test_pearson
0,MSRpar,0.969305,0.644150,0.583520
1,MSRvid,0.978473,0.736964,0.805225
2,SMTeuroparl,0.960448,0.749384,0.481851
3,all,0.982601,0.823676,0.770720


Here, we see that while validation performance is largely the same, test performance is worse. This suggests that n-grams are a key feature that generalizes well to the test set.

## Final Model

Now we train a final model using all features and all training data, in hope of achieving the best possible performance on the full test set, including unseen datasets.

In [22]:
fully_trained_model = RandomForestRegressor()
fully_trained_model.fit(all_train_df[feature_names], all_train_df.gs)

RandomForestRegressor()

In [24]:
def evaluate_model_across_test(model, selected_features):
    results = []
    datasets = test_datasets
    for dataset in datasets:
        dataset_results = []
        dataset_results.append(dataset)

        preds = model.predict(all_test_df[all_test_df.dataset == dataset][selected_features])
        dataset_results.append(pearsonr(all_test_df[all_test_df.dataset == dataset].gs, preds)[0])

        results.append(dataset_results)

    preds = model.predict(all_test_df[selected_features])
    results.append(['all', pearsonr(all_test_df.gs, preds)[0]])
    results = pd.DataFrame(results, columns=["dataset", "test_pearson"])
    return results

test_results = evaluate_model_across_test(fully_trained_model, feature_names)
test_results

,dataset,test_pearson
0,MSRpar,0.640364
1,MSRvid,0.844194
2,SMTeuroparl,0.496497
3,OnWN,0.641504
4,SMTnews,0.441386
5,all,0.733428


In [25]:
results_to_beat = pd.DataFrame(np.array([[.683, .873, .528, .664, .493, 0.823]]).T, index=[*test_datasets, 'all'], columns=["pearson_to_beat"])
results_to_beat

,pearson_to_beat
MSRpar,0.683
MSRvid,0.873
SMTeuroparl,0.528
OnWN,0.664
SMTnews,0.493
all,0.823


# Discussion